In [ ]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
load_dotenv(dotenv_path='D:/GenAI-Practice/AgenticAI-Projects/SmartAssistant/.env')

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                             api_key=os.getenv('GOOGLE_API_KEY'))

In [ ]:
schemas = [
    ResponseSchema(name="query_type", description="The category of query. Should be one of: rag, db, llm."),
    ResponseSchema(name="reason", description="Reason why the query belongs to this category.")
]

parser = StructuredOutputParser.from_response_schemas(schemas)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_template(
    """
    You are a query classification expert.
    Given a user's query, classify it into one of the following:
    - rag → if it needs knowledge from uploaded documents
    - db → if it needs database lookup or business data
    - llm → if it’s a general question not needing external data.

    Query: {query}

    {format_instructions}
    """
)

In [ ]:
messages = prompt.format_messages(query="Can you summarize this section from document?", format_instructions=format_instructions)
response = llm(messages)

print(f"response \n{response}")

In [ ]:
response.keys

### testing the SQL agent

In [ ]:
import os
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_models import ChatHuggingFace
from transformers import pipeline
from langchain_groq import ChatGroq

from dotenv import load_dotenv
load_dotenv(dotenv_path='D:/GenAI-Practice/AgenticAI-Projects/SmartAssistant/.env')

d:\GenAI-Practice\env_RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# Load database
db = SQLDatabase.from_uri("sqlite:///D:/GenAI-Practice/AgenticAI-Projects/SmartAssistant/app/data/uploads/bank_domain.db")

# Create toolkit for SQL agent
# LLM = ChatGoogleGenerativeAI(model="gemini-2.5-pro", api_key=os.getenv('GOOGLE_API_KEY'))
# pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta")
# LLM = ChatHuggingFace(pipeline=pipe)
# llm=ChatGroq(model="llama3-70b-8192")
LLM = ChatGroq(
    temperature = 0,
    groq_api_key = os.getenv("groq_api_key"),
    model_name = os.getenv("llama_model_name")
)
toolkit = SQLDatabaseToolkit(db=db, llm=LLM)

# Create the SQL agent
agent_executor = create_sql_agent(
    llm=toolkit.llm,
    toolkit=toolkit,
    verbose=True,
    agent_type="openai-tools",  # works well with Gemini too
)

# Run query
query = "Total how many branch locations are there?"
response = agent_executor.invoke({"input": query})

# Update chatbot state
answer = response["output"]



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded:   Then I should double check my query before running it to get the answer.



accounts, branch_locations, credit_cards, credit_exposures, customers, employees, insurance_policies, investments, loans, transactions
Invoking: `sql_db_schema` with `{'table_names': 'branches, branch_locations'}`
responded:   Then I should double check my query before running it to get the answer.



Error: table_names {'branches'} not found in database
Invoking: `sql_db_query_checker` with `{'query': 'SELECT COUNT(*) FROM branch_locations;'}`
responded:   Then I should double check my query before running it to get the answer.



```sql
SELECT COUNT(*) FROM branch_locations;
```
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM branch_locations;'}`
responded:   Then I should double check my query before running it to get the answer.



[(10,)]There are 10 branch locations.

> Finish

In [3]:
answer

'There are 10 branch locations.'